## **CLASE 2: Middleware, rutas protegidas y autorización basada en roles**

### Objetivos de aprendizaje:

* Proteger rutas con `Depends` en FastAPI usando JWT.
* Controlar acceso según el rol del usuario (`admin`, `user`).
* Implementar middleware personalizado y pruebas de acceso.

---


### Contenidos y desarrollo


#### Árbol de carpetas para el proyecto FastAPI

```plaintext
my_project/
│
├── app/
│   ├── main.py
│   ├── config.py
│
│   ├── auth/
│   │   ├── jwt_handler.py         # Creación de tokens JWT
│   │   ├── security.py            # Validación y extracción del usuario actual
│   │   └── roles.py               # Verificación de rol por usuario
│
│   ├── routers/
│   │   ├── users.py               # Ruta protegida para obtener datos del usuario
│   │   └── admin.py               # Ruta protegida para administradores
│
│   ├── middleware/
│   │   └── validate_token.py      # Middleware que exige token en rutas /admin

```

---



#### 1. **Definir las variables de entorno**

app/config.py

In [ ]:
SECRET_KEY = "mi_clave_secreta"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30


#### 2. **Autenticación básica y creación de JWT seguro**

app/auth/jwt_handler.py

In [ ]:
from jose import JWTError, jwt
from datetime import datetime, timedelta
from app.config import SECRET_KEY, ALGORITHM, ACCESS_TOKEN_EXPIRE_MINUTES

def create_access_token(data: dict, expires_delta: timedelta = None):
    to_encode = data.copy()
    expire = datetime.now() + (expires_delta or timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES))
    to_encode.update({"exp": expire})
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)


> **Buenas prácticas**:
>
> * Nunca exponer la clave secreta (`SECRET_KEY`) en el código fuente.
> * Establecer expiración del token.

---


#### 3. **Rutas protegidas y extracción del token con `Depends`**

#### Conceptos clave previos

| Concepto                 | Descripción                                                                                                                                 |
| ------------------------ | ------------------------------------------------------------------------------------------------------------------------------------------- |
| **JWT (JSON Web Token)** | Es un token seguro que contiene datos como el usuario y su rol. Se firma digitalmente y tiene fecha de expiración.                          |
| **`Depends` de FastAPI** | Mecanismo para declarar dependencias (como funciones reutilizables) que pueden validar, transformar o autorizar peticiones automáticamente. |
| **OAuth2PasswordBearer** | Clase de FastAPI que extrae automáticamente el token desde el encabezado HTTP `Authorization: Bearer <token>`.                              |

---


#### Paso a paso del código



* **Importaciones**


app/auth/security.py

In [ ]:
from fastapi import Depends, HTTPException, status
from fastapi.security import OAuth2PasswordBearer
from jose import JWTError, jwt
from app.config import SECRET_KEY, ALGORITHM

* **Importamos los módulos**:

  * `Depends`: para inyectar funciones como dependencias.
  * `OAuth2PasswordBearer`: para extraer el token desde el encabezado.
  * `jwt.decode`: para leer el contenido del token.
  * `HTTPException`: para lanzar errores controlados.

---


* **Crear esquema de extracción del token**



In [ ]:
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="login")

* Esto configura FastAPI para esperar un token JWT en el **encabezado Authorization** usando el esquema Bearer:

  ```http
  Authorization: Bearer <token>
  ```
* `tokenUrl="login"` le dice a FastAPI que el endpoint de autenticación (que genera el token) está en `/login`.

---


* **Crear función de dependencia para obtener usuario actual**



In [ ]:
def get_current_user(token: str = Depends(oauth2_scheme)):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        print("Payload recibido:", payload)  # <-- Agrega esto
        user = payload.get("sub")
        role = payload.get("role")
        print("user:", user, "role:", role)  # <-- Agrega esto
        if user is None or role is None:
            raise credentials_exception()
        return {"username": user, "role": role}
    except JWTError:
        raise credentials_exception()

#### ¿Qué hace esta función?

1. **Extrae el token** automáticamente del encabezado usando `Depends(oauth2_scheme)`.
2. **Decodifica** el token con la clave secreta y el algoritmo configurado.
3. Verifica que en el `payload` haya un `sub` (username) y un `role`.
4. Si el token es inválido o no tiene esos datos, lanza una excepción `401 Unauthorized`.

> Esta función puede ser utilizada como dependencia en cualquier endpoint que requiera autenticación.

---


* **Manejo de errores personalizados**



In [ ]:
def credentials_exception():
    return HTTPException(
        status_code=status.HTTP_401_UNAUTHORIZED,
        detail="Credenciales inválidas",
        headers={"WWW-Authenticate": "Bearer"},
    )



* Lanza un error `401 Unauthorized` con mensaje claro.
* El header `WWW-Authenticate` es estándar para OAuth2 y permite a clientes como SwaggerUI solicitar token nuevamente.

---


#### 4. **Autorización basada en roles**

app/auth/roles.py

In [ ]:
from fastapi import Depends, HTTPException
from app.auth.security import get_current_user

def require_role(required_role: str):
    def role_dependency(user: dict = Depends(get_current_user)):
        if user["role"] != required_role:
            raise HTTPException(status_code=403, detail="Acceso denegado")
        return user
    return role_dependency


---

#### 5. **Middleware personalizado para validación de token**

app/middleware/validate_token.py

In [ ]:
# Importamos la clase base para crear middlewares personalizados en Starlette (usado por FastAPI)
from starlette.middleware.base import BaseHTTPMiddleware

# Importamos la clase Request para manejar solicitudes HTTP en FastAPI/Starlette
from starlette.requests import Request

# Importamos JSONResponse para devolver respuestas en formato JSON
from fastapi.responses import JSONResponse


# Middleware personalizado para validar si un endpoint requiere token de autorización
class ValidateTokenMiddleware(BaseHTTPMiddleware):

    # El método dispatch se ejecuta cada vez que se recibe una solicitud HTTP
    async def dispatch(self, request: Request, call_next):
        """
        request: Objeto que contiene toda la información de la solicitud HTTP.
        call_next: Función que permite pasar la solicitud al siguiente middleware o endpoint.
        """

        # Si la ruta solicitada comienza con "/admin", requerimos validación de token
        if request.url.path.startswith("/admin"):
            # Obtenemos el valor del encabezado Authorization
            token = request.headers.get("Authorization")

            # Si no se envió el token en la solicitud, respondemos con un error 401 (No autorizado)
            if token is None:
                return JSONResponse(
                    status_code=401,  # Código HTTP de No autorizado
                    content={"detail": "Token requerido"}  # Mensaje de error en formato JSON
                )

        # Si la ruta no es /admin o si el token está presente, continuamos con la solicitud
        return await call_next(request)


---

#### 6. **Ruta protegida para obtener datos del usuario**

app/routers/users.py

In [ ]:
from fastapi import APIRouter, Depends
from app.auth.security import get_current_user

router = APIRouter()

@router.get("/datos-usuario")
def get_user_data(current_user: dict = Depends(get_current_user)):
    return {"usuario_actual": current_user["username"], "rol": current_user["role"]}


---

#### 7. **Ruta protegida para administradores**

app/routers/admin.py

In [ ]:
from fastapi import APIRouter, Depends
from app.auth.roles import require_role

router = APIRouter()

@router.get("/admin-data")
def get_admin_data(current_user=Depends(require_role("admin"))):
    return {"msg": "Datos solo para administradores"}


app/main.py

In [ ]:
from fastapi import FastAPI
from app.routers import users, admin
from app.middleware.validate_token import ValidateTokenMiddleware

app = FastAPI()

# Middleware personalizado
app.add_middleware(ValidateTokenMiddleware)

# Routers
app.include_router(users.router)
app.include_router(admin.router)


---

#### 8. **Instala las dependencias**

Desde la raíz del proyecto (`my_project/`), crea el archivo `requirements.txt` con lo siguiente:

```txt
fastapi
uvicorn
python-jose[cryptography]
passlib[bcrypt]
```

Luego, instala todo:

```bash
pip install -r requirements.txt
```

---

#### 9. **Ejecutar la aplicación**

Desde la raíz del proyecto:

```bash
uvicorn app.main:app --reload
```

Esto:

* Levanta el servidor en `http://127.0.0.1:8000`
* El parámetro `--reload` reinicia el servidor automáticamente si haces cambios

---

#### 10. **Simular un token JWT** - simulacion.py

Como aún no hay un endpoint de login real, puedes **simular un token manualmente** desde Python:




In [ ]:
# ejecuta esto en un archivo Python aparte o consola interactiva
from jose import jwt
from datetime import datetime, timedelta

SECRET_KEY = "clave_super_segura"
ALGORITHM = "HS256"

data = {
    "sub": "juanperez",
    "role": "admin",  # o "user"
    "exp": datetime.now() + timedelta(minutes=30)
}

token = jwt.encode(data, SECRET_KEY, algorithm=ALGORITHM)
print(token)

# Copiar el token generado y usarlo en Postman/Docs con:

#Authorization: Bearer <token_aquí>



---

#### 11. **Probar desde Postman**

Ahora que tienes un token válido, puedes usarlo para probar tus rutas protegidas.

Abre Postman y sigue estos pasos:

1. Crear una solicitud `GET`

Por ejemplo, para el endpoint:

```
GET http://127.0.0.1:8000/datos-usuario
```

---

2. Ir a la pestaña **Authorization**

* Tipo: **Bearer Token**
* En el campo "Token", pega el token (solo el valor, sin escribir "Bearer")

Ejemplo:

```
eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9...
```

---

3. Haz clic en **Send**

Postman enviará la petición con el token en el encabezado:

```
Authorization: Bearer <token>
```

---

Resultado esperado

Si el token es válido:

* Respuesta 200 OK
* Verás algo como:

```json
{
  "usuario_actual": "juanperez",
  "rol": "admin"
}
```





#### 12. **Pruebas que puedes hacer**

| Escenario                                   | Qué probar              | Resultado esperado       |
| ------------------------------------------- | ----------------------- | ------------------------ |
| Acceder a `/datos-usuario` sin token        | Lanza error 401         | "Credenciales inválidas" |
| Acceder a `/datos-usuario` con token válido | Devuelve username y rol | 200 OK                   |
| Acceder a `/admin-data` con rol `admin`     | Acceso permitido        | 200 OK                   |
| Acceder a `/admin-data` con rol `user`      | Lanza error 403         | "Acceso denegado"        |
| Acceder a `/admin-data` sin token           | Middleware lanza 401    | "Token requerido"        |

---


## Buenas prácticas de seguridad

| Recomendación                                    | Justificación                                              |
| ------------------------------------------------ | ---------------------------------------------------------- |
| Validar `exp` (expiración) del token             | Evita el uso de tokens viejos.                             |
| Usar HTTPS en producción                         | Protege el token en tránsito.                              |
| No incluir información sensible en el token      | El contenido de JWT puede ser leído por cualquier persona. |
| Usar `sub`, `role`, y `exp` como claims estándar | Facilita la interoperabilidad y mantenimiento del código.  |

---
